In [1]:
import pandas as pd

# Read the CSV and Perform Basic Data Cleaning

In [2]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


In [3]:
len(df)

6991

# Select your features (columns)

In [4]:
# Set features. This will also be used as your x values.
X = df[['koi_period', 'koi_teq', 'koi_insol', 'koi_model_snr', 'koi_duration', 'koi_impact', 'koi_depth', 'koi_prad']]
y = df['koi_disposition']
X.head()

,koi_period,koi_teq,koi_insol,koi_model_snr,koi_duration,koi_impact,koi_depth,koi_prad
0,54.418383,443,9.11,25.8,4.50700,0.586,874.8,2.83
1,19.899140,638,39.30,76.3,1.78220,0.969,10829.0,14.60
2,1.736952,1395,891.96,505.6,2.40641,1.276,8079.2,33.46
3,2.525592,1406,926.16,40.9,1.65450,0.701,603.3,2.75
4,4.134435,1160,427.65,40.2,3.14020,0.762,686.0,2.77


# Create a Train Test Split

Use `koi_disposition` for the y values

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [6]:
len(X_test)

1748

# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [7]:
# Scale your data
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train the Model



In [8]:
from sklearn.linear_model import LogisticRegression
classifier= LogisticRegression()
classifier

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [9]:
classifier.fit(X_train_scaled, y_train)

C:\Users\ashka\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\ashka\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [10]:
print(f"Training Data Score: {classifier.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test_scaled, y_test)}")

Training Data Score: 0.5708563799351516
Testing Data Score: 0.5600686498855835


In [11]:
predictions = classifier.predict(X_test_scaled)

In [12]:
dd = pd.DataFrame({"Prediction": predictions, "Actual":y_test})
dd.head()

,Prediction,Actual
4990,FALSE POSITIVE,FALSE POSITIVE
1425,CONFIRMED,CONFIRMED
144,FALSE POSITIVE,CONFIRMED
6589,FALSE POSITIVE,FALSE POSITIVE
4443,CONFIRMED,CANDIDATE


In [13]:
# To Confirm the accuracy of testing data
s = dd["Prediction"].count()
wrong = 0
for i in range(0,s):
    a=dd["Prediction"].iloc[i]
    b=dd["Actual"].iloc[i]
    if a != b:
        wrong = wrong + 1
        
(s - wrong) / s

0.5600686498855835

# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [14]:
# Create the GridSearchCV model
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [0.5, 1, 2, 5, 10, 100], 'solver' : ['newton-cg', 'lbfgs', 'sag', 'saga', 'liblinear']}

grid = GridSearchCV(classifier, param_grid, verbose = 3)

In [15]:
# Train the model with GridSearch
grid.fit(X_train_scaled, y_train)

C:\Users\ashka\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


Fitting 3 folds for each of 30 candidates, totalling 90 fits
[CV] C=0.5, solver=newton-cg .........................................
[CV] ............. C=0.5, solver=newton-cg, score=0.512, total=   0.0s
[CV] C=0.5, solver=newton-cg .........................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Users\ashka\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
C:\Users\ashka\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
C:\Users\ashka\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\ashka\Anaconda3\lib\site-pa

[CV] ............. C=0.5, solver=newton-cg, score=0.513, total=   0.0s
[CV] C=0.5, solver=newton-cg .........................................
[CV] ............. C=0.5, solver=newton-cg, score=0.513, total=   0.0s
[CV] C=0.5, solver=lbfgs .............................................
[CV] ................. C=0.5, solver=lbfgs, score=0.512, total=   0.0s
[CV] C=0.5, solver=lbfgs .............................................
[CV] ................. C=0.5, solver=lbfgs, score=0.513, total=   0.0s
[CV] C=0.5, solver=lbfgs .............................................
[CV] ................. C=0.5, solver=lbfgs, score=0.513, total=   0.1s
[CV] C=0.5, solver=sag ...............................................


C:\Users\ashka\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\ashka\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\ashka\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\ashka\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


...


[CV] C=100, solver=saga ..............................................
[CV] .................. C=100, solver=saga, score=0.627, total=   0.2s

C:\Users\ashka\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\ashka\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\ashka\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)



[CV] C=100, solver=liblinear .........................................
[CV] ............. C=100, solver=liblinear, score=0.611, total=   0.0s
[CV] C=100, solver=liblinear .........................................
[CV] ............. C=100, solver=liblinear, score=0.628, total=   0.0s
[CV] C=100, solver=liblinear .........................................
[CV] ............. C=100, solver=liblinear, score=0.631, total=   0.0s


C:\Users\ashka\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\ashka\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
[Parallel(n_jobs=1)]: Done  90 out of  90 | elapsed:    5.9s finished
C:\Users\ashka\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='warn',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='warn',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'C': [0.5, 1, 2, 5, 10, 100],
                         'solver': ['newton-cg', 'lbfgs', 'sag', 'saga',
                                    'liblinear']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=3)

In [16]:
print(grid.best_params_)
print(grid.best_score_)

{'C': 100, 'solver': 'newton-cg'}
0.6231165363341599


In [17]:
predictions = grid.predict(X_test_scaled)

In [18]:
target_names = ["False Positive", "Candidate", "Confirmed"]

In [19]:
from sklearn.metrics import classification_report
print(classification_report(y_test, predictions,
                            target_names=target_names))

                precision    recall  f1-score   support

False Positive       0.40      0.05      0.08       420
     Candidate       0.50      0.70      0.58       461
     Confirmed       0.70      0.85      0.76       867

      accuracy                           0.61      1748
     macro avg       0.53      0.53      0.48      1748
  weighted avg       0.57      0.61      0.55      1748



In [20]:
print(f"Training Data Score: {grid.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {grid.score(X_test_scaled, y_test)}")

Training Data Score: 0.6261682242990654
Testing Data Score: 0.61441647597254


In [21]:
gg = pd.DataFrame({"Prediction": predictions, "Actual":y_test})
gg.head()

,Prediction,Actual
4990,FALSE POSITIVE,FALSE POSITIVE
1425,CONFIRMED,CONFIRMED
144,FALSE POSITIVE,CONFIRMED
6589,FALSE POSITIVE,FALSE POSITIVE
4443,CONFIRMED,CANDIDATE


In [22]:
# To confirm accuracy
s = gg["Prediction"].count()
wrong = 0
for i in range(0,s):
    a=gg["Prediction"].iloc[i]
    b=gg["Actual"].iloc[i]
    if a != b:
        wrong = wrong + 1
        
(s - wrong) / s

0.61441647597254